# KoBART Model 성능 평가 및 비교
KoBART 모델이 무료 클라우드인 오라클 프리티어 서버에 배포하기에는 무거운 모델이라서 KoBART small 모델을 개발하여 성능을 비교했다.
하지만, KoBART small 모델과의 요약속도 차이가 드라마틱하게 달라지진 않고, 성능 또한 기존 KoBART 모델이 훨씬 좋다고 판단하여 결국 KoBART 모델을 사용했다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.11.0+cu113
cuda version: 11.3
cudnn version:8200


In [ ]:
cd /content/drive/MyDrive/AI_camp/CP1/KoBART-summarization

/content/drive/MyDrive/AI_camp/CP1/KoBART-summarization


In [ ]:
# 파인 튜닝된 KoBART 모델 불러오기 to use GPU
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

device = torch.device("cuda")
model = BartForConditionalGeneration.from_pretrained('./kobart_summary')
model.load_state_dict(torch.load('/content/drive/MyDrive/AI_camp/CP1/KoBART-summarization/kobart_summary/pytorch_model.bin'))
model.to(device)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

In [ ]:
DATA_PATH = '/content/drive/.....'

In [ ]:
# 성능평가용 데이터 1000개
import pandas as pd

v = pd.read_csv(f'{DATA_PATH}test2.tsv', sep= '\t')
valid_data = v.sample(n=1000, random_state = 42)
display(valid_data.head())

,news,summary
2915,국립대구과학관(관장 김주한)은 인류 달 탐사 50주년과 국제천문연맹 설립 100주년...,인류 최초 닐 암스트롱의 달 탐사 50주년과 국제 천문연맹 설립 100주년을 맞이하...
19964,"산재보험 사각지대에 놓여 있는 방문·돌봄서비스 종사자, 화물차주, 정보통신(IT) ...",산재보험 사각지대에 있는 돌봄서비스 종사자와 자유계자자 등도 산재보험을 받을 수 있...
5264,이너테인먼트 바스타라이브VR 서비스를 이용해 머스트비 VR 팬미팅을 진행하고 있다....,31일 이너테인먼트는 지난 27일 국내 인기 아이돌 머스트비와 아바타로 접속한 팬들...
13328,넷마블(대표 권영식)은 모바일 실시간 전략 대전 게임 '매직: 마나스트라이크'의 플...,넷마블은 세계 최초의 트레이딩 카드 게임인 '매직: 더 개더링'의 IP를 고품질 3...
16429,올해 1분기 수출이 전년 같은 기간에 비해 줄었다.수출이 전년도 동기에 비해서 감소...,지난 8일 한국은행이 발표한 '2019년 3월 국제수지'에 의하면 올 1분기 수출은...


In [ ]:
# 문서를 토크나이즈 하고, Fine tuning 된 KoBART 모델이 문서를 요약함
# 요약된 문서는 성능 평가를 위해 output_list라는 리스트에 저장
%%time
output_list = []
for i in valid_data['news']:
    input_ids = tokenizer.encode(i)
    input_ids = torch.tensor(input_ids, device = torch.device('cuda'))
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=64, num_beams=2)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    output_list.append(output)

CPU times: user 4min 54s, sys: 665 ms, total: 4min 55s
Wall time: 4min 54s


In [ ]:
# 경량화를 위해 만들었던 KoBART small 모델도 불러오기
tokenizer_small = PreTrainedTokenizerFast.from_pretrained('cosmoquester/bart-ko-small')
device = torch.device("cuda")
model_small = torch.load('/content/drive/MyDrive/AI_camp/CP1/KoBART-small/model.pt')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
#KoBART small 모델을 사용한 문서요약 예측 함수
def predict(text):
    text = text.replace('\n', ' ')
    raw_input_ids = tokenizer_small.encode(text)
    input_ids = [tokenizer_small.bos_token_id] + raw_input_ids + [tokenizer_small.eos_token_id]
    summary_ids = model_small.generate(torch.tensor([input_ids], dtype=torch.int32, device=device),  num_beams=2,  max_length=64,  eos_token_id=tokenizer_small.eos_token_id)
    return tokenizer_small.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

In [ ]:
#KoBART small 모델로 예측
%%time
output_list2 = []
for i in valid_data['news']:
    output_list2.append(predict(i))

CPU times: user 4min 26s, sys: 674 ms, total: 4min 26s
Wall time: 4min 26s


In [ ]:
#Fine tuning한 KoBART 모델과 KoBART small 모델 성능 비교.
#!pip install rouge
from rouge import Rouge

rouge = Rouge()
print('기존 KoBART 모델', rouge.get_scores(output_list,valid_data.summary.tolist(), avg=True))

print('')

print('KoBART-small 모델', rouge.get_scores(output_list2,valid_data.summary.tolist(), avg=True))
#Rouge score: 모델이 요약한 문서와 사람이 요약한 문서를 비교해서 중요한 단어가 얼마나 겹치는 가를 기반으로 성능을 평가한다.
# small보다 기존의 KoBART 모델의 성능이 더 뛰어난 것을 확인할 수 있다.

기존 KoBART 모델 {'rouge-1': {'r': 0.283769893592443, 'p': 0.33985915133698996, 'f': 0.3001045641284538}, 'rouge-2': {'r': 0.16390030621293916, 'p': 0.2018989156577948, 'f': 0.17502724363967237}, 'rouge-l': {'r': 0.25873905117159457, 'p': 0.31167202652406834, 'f': 0.2743092474260298}}

KoBART-small 모델 {'rouge-1': {'r': 0.2509237634264106, 'p': 0.30300402719159075, 'f': 0.26840430464543547}, 'rouge-2': {'r': 0.14028444998357678, 'p': 0.17043379746868575, 'f': 0.1504494574030706}, 'rouge-l': {'r': 0.24060665033213777, 'p': 0.29072046470719126, 'f': 0.25743577316637756}}


In [ ]:
# 실제 요약문과 KoBART 모델의 요약문 비교
print('KoBART 요약: ', output_list[10])
print('정답: ', valid_data.summary.iloc[10])

KoBART 요약:  한국감정원은 25일 GTX A노선 민간투자사업의 토지보상계획을 공고함에 따라 2023년 개통을 목표로 보상이 완료되는 토지부터 개발에 착수한다는 계획이다.
정답:  한국감정원은 25일 GTX A노선 개발과 관련한 토지보상 절차가 개시되어 사업 본격화가 초읽기에 들어가 계획을 공고하고  8월께 감정평가와 9월에는 협의통지 등을 순차적으로 진행할 예정이다.


In [ ]:
# 실제 요약문과 KoBART small모델의 요약문 비교
print('KoBART-small 요약: ', output_list2[10])
print('정답: ', valid_data.summary.iloc[10])

KoBART-small 요약:  수도권광역급행철도-A노선 개발과 관련한 토지보상 절차가 개시됐다. 경기도 303필지와 서울시 80필지 등 편입면적 29만8044.3m2가 대상이다.국토교통부는 토지보상 공고를 시작
정답:  한국감정원은 25일 GTX A노선 개발과 관련한 토지보상 절차가 개시되어 사업 본격화가 초읽기에 들어가 계획을 공고하고  8월께 감정평가와 9월에는 협의통지 등을 순차적으로 진행할 예정이다.
